In [1]:
from run_capsule import *
from decoding_utils import *
import sys
sys.argv[1:] = ['--result_prefix', 'debug']
params = Params()

In [12]:
(
        utils.get_df('units', lazy=True)
    # .filter(pl.col('session_id') == "733891_2024-09-16")
        .filter(
            # pl.col('session_id').is_in(session_ids),
            params.units_query,
        )
        .pipe(group_structures, keep_originals=True)
        .pipe(repeat_multi_probe_areas)
        .filter(
            pl.col('unit_id').n_unique().ge(params.n_units).over(params.units_group_by)
        )
        .select(*params.units_group_by, 'unit_id')
    .group_by('session_id', 'structure', 'electrode_group_names')
    .agg(pl.col('unit_id').len().alias('n_units'))
        # .unique(params.units_group_by)
    # .sort('structure', 'electrode_group_names')
    .sort('n_units')
).collect()

session_id,structure,electrode_group_names,n_units
str,str,list[str],u32
"""664851_2023-11-14""","""SCig""","[""probeD""]",25
"""703333_2024-04-11""","""FRP""","[""probeE""]",25
"""713655_2024-08-07""","""TH""","[""probeA"", ""probeC""]",25
"""741148_2024-10-17""","""PRE""","[""probeD""]",25
"""636397_2022-09-26""","""CA3""","[""probeC""]",25
…,…,…,…
"""733780_2024-08-27""",null,"[""probeD""]",2837
"""733780_2024-08-27""",null,"[""probeB""]",2837
"""733780_2024-08-27""",null,"[""probeC"", ""probeB"", … ""probeA""]",2837


In [5]:
df.sort(pl.col('electrode_group_names').list.len())

session_id,structure,electrode_group_names
str,str,list[str]
"""686176_2023-12-07""","""DG""","[""probeD"", ""probeD"", … ""probeD""]"
"""660023_2023-08-09""","""SCs""","[""probeC"", ""probeC"", … ""probeC""]"
"""726088_2024-06-20""","""NOT""","[""probeB"", ""probeB"", … ""probeB""]"
"""713655_2024-08-07""","""TH""","[""probeC"", ""probeA"", … ""probeC""]"
"""708016_2024-04-29""","""DG""","[""probeE"", ""probeE"", … ""probeD""]"
…,…,…
"""670248_2023-08-02""",null,"[""probeA"", ""probeA"", … ""probeF""]"
"""644864_2023-02-01""",null,"[""probeA"", ""probeA"", … ""probeF""]"
"""744740_2024-11-13""",null,"[""probeA"", ""probeA"", … ""probeE""]"


In [24]:
(
    utils.get_df('units', lazy=True)
         # .filter(pl.col('session_id') == "668759_2023-07-13")
    .filter(
        # pl.col('session_id').is_in(session_ids),
        params.units_query,
    )
    .pipe(group_structures, keep_originals=True)
    .pipe(repeat_multi_probe_areas)
    .select('session_id', 'structure', 'electrode_group_names', 'unit_id')
    .filter(
        pl.col('unit_id').n_unique().ge(params.n_units).over(params.units_group_by)
    )
    .select(params.units_group_by)
    .unique(params.units_group_by)
    .sort(pl.col('electrode_group_names').list.len())
    .collect()
)

session_id,structure,electrode_group_names
str,str,list[str]
"""708016_2024-04-29""","""DG""","[""probeE"", ""probeE"", … ""probeD""]"
"""660023_2023-08-09""","""SCs""","[""probeC"", ""probeC"", … ""probeC""]"
"""713655_2024-08-07""","""TH""","[""probeC"", ""probeA"", … ""probeC""]"
"""726088_2024-06-20""","""NOT""","[""probeB"", ""probeB"", … ""probeB""]"
"""686176_2023-12-07""","""DG""","[""probeD"", ""probeD"", … ""probeD""]"
…,…,…
"""670248_2023-08-02""",null,"[""probeA"", ""probeA"", … ""probeF""]"
"""644864_2023-02-01""",null,"[""probeA"", ""probeA"", … ""probeF""]"
"""744740_2024-11-13""",null,"[""probeA"", ""probeA"", … ""probeE""]"


In [75]:
 (
        utils.get_df('units', lazy=True)
     .filter(pl.col('session_id') == "668759_2023-07-13")
 
        # create list of probe names per session-structure:
        .group_by('session_id', 'structure')
        .agg(
            pl.all().exclude('electrode_group_name'),
            pl.col('electrode_group_name').unique().alias('electrode_group_names'),
        )
        # duplicate those with multiple probes (col becomes List[List[String]]):
        .with_columns(
            pl.when(pl.col('electrode_group_names').list.n_unique().gt(1))
            .then(pl.col('electrode_group_names').repeat_by(2))
            .otherwise(pl.col('electrode_group_names').repeat_by(1))
        )
        # # .explode(pl.all().exclude('session_id', 'structure', 'electrode_group_names'))
        .explode('electrode_group_names')

        # convert the duplicated rows to joined 'probeA_probeB' format:
        .with_columns(
            pl.when(pl.col('electrode_group_names').is_first_distinct().over('session_id', 'structure'))
            .then(pl.col('electrode_group_names'))
            .otherwise(pl.col('electrode_group_names').list.join('_').cast(pl.List(pl.String)))
        )
        # create individual lists vs single list for duplicate/non-duplicate rows:
        # [probeA, probeB] vs [[probeA], [probeB]]
        .with_columns(
            pl.col('electrode_group_names').list.eval(pl.element().str.split('_'))
        )
         .explode(pl.all().exclude('session_id', 'structure', 'electrode_group_names'))
        .explode('electrode_group_names')
    
         .select('structure', 'electrode_group_names', 'unit_id')
         .sort('unit_id')

).collect()


structure,electrode_group_names,unit_id
str,list[str],str
"""RT""","[""probeB""]","""668759_2023-07-13_B-0"""
"""RT""","[""probeB""]","""668759_2023-07-13_B-1"""
"""VAL""","[""probeB""]","""668759_2023-07-13_B-10"""
"""VAL""","[""probeB""]","""668759_2023-07-13_B-100"""
"""VPM""","[""probeB""]","""668759_2023-07-13_B-101"""
…,…,…
"""MOs""","[""probeD"", ""probeE"", ""probeF""]","""668759_2023-07-13_F-98"""
"""MOs""","[""probeD""]","""668759_2023-07-13_F-99"""
"""MOs""","[""probeE""]","""668759_2023-07-13_F-99"""


In [ ]:
 (
        utils.get_df('units', lazy=True)
        # create list of probe names per session-structure:
        .group_by('session_id', 'structure')
        .agg(
            pl.all().exclude('electrode_group_name'),
            pl.col('electrode_group_name').alias('electrode_group_names'),
        )
        # duplicate those with multiple probes (col becomes List[List[String]]):
        .with_columns(
            pl.when(pl.col('electrode_group_names').list.n_unique().gt(1))
            .then(pl.col('electrode_group_names').repeat_by(2))
            .otherwise(pl.col('electrode_group_names').repeat_by(1))
        )
        .explode(pl.all().exclude('session_id', 'structure', 'electrode_group_names'))
        .explode('electrode_group_names')
        # convert the duplicated rows to joined 'probeA_probeB' format:
        .with_columns(
            pl.when(pl.col('electrode_group_names').is_first_distinct().over('session_id', 'structure'))
            .then(pl.col('electrode_group_names'))
            .otherwise(pl.col('electrode_group_names').list.join('_').cast(pl.List(pl.String)))
        )
        # create individual lists vs single list for duplicate/non-duplicate rows:
        # [probeA, probeB] vs [[probeA], [probeB]]
        .with_columns(
            pl.col('electrode_group_names').list.eval(pl.element().str.split('_'))
        )
        .explode('electrode_group_names')
    ).collect()

In [14]:
(
        df
        # create list of probe names per session-structure:
        .group_by('session_id', 'structure')
        .agg(
            pl.all().exclude('electrode_group_name'),
            pl.col('electrode_group_name').alias('electrode_group_names'),
        )
        # duplicate those with multiple probes (col becomes List[List[String]]):
        .with_columns(
            pl.when(pl.col('electrode_group_names').list.n_unique().gt(1))
            .then(pl.col('electrode_group_names').repeat_by(2))
            .otherwise(pl.col('electrode_group_names').repeat_by(1))
        )
        .explode('electrode_group_names')
        # convert the duplicated rows to joined 'probeA_probeB' format:
        .with_columns(
            pl.when(pl.col('electrode_group_names').is_first_distinct().over('session_id', 'structure'))
            .then(pl.col('electrode_group_names'))
            .otherwise(pl.col('electrode_group_names').list.join('_').cast(pl.List(pl.String)))
        )
        # create individual lists vs single list for duplicate/non-duplicate rows:
        # [probeA, probeB] vs [[probeA], [probeB]]
        .with_columns(
            pl.col('electrode_group_names').list.eval(pl.element().str.split('_'))
        )
        .explode('electrode_group_names')
    )

session_id,structure,electrode_group_names
str,str,list[str]
"""681532_2023-10-16""","""VISp""","[""probeB""]"
"""681532_2023-10-16""","""VISp""","[""probeB""]"
"""681532_2023-10-16""","""VISp""","[""probeB""]"
"""681532_2023-10-16""","""VISp""","[""probeB""]"
"""681532_2023-10-16""","""VISp""","[""probeB""]"
…,…,…
"""733891_2024-09-16""","""DG""","[""probeC""]"
"""733891_2024-09-16""","""DG""","[""probeC""]"
"""733891_2024-09-16""","""DG""","[""probeC""]"


In [4]:
Params(
    result_prefix='debug', session_id='703333_2024-04-10', run_id='20250327_165658', override_params_json='{}', skip_existing=True,
).model_dump_json()

'{"result_prefix":"debug","run_id":"20250327_165658","session_table_query":"is_ephys & is_task & is_annotated & is_production & issues==\'[]\'","unit_criteria":"medium","n_repeats":25,"input_data_type":"spikes","vid_angle_facemotion":"face","vid_angle_LP":"behavior","central_section":"4_blocks_plus","exclude_cue_trials":false,"n_unit_threshold":5,"keep_n_SVDs":500,"LP_parts_to_keep":["ear_base_l","eye_bottom_l","jaw","nose_tip","whisker_pad_l_side"],"spikes_binsize":0.2,"spikes_time_before":0.2,"spikes_time_after":0.01,"use_structure_probe":true,"crossval":"5_fold","labels_as_index":true,"decoder_type":"LogisticRegression","only_use_all_units":false,"predict":"context","regularization":null,"penalty":null,"solver":null,"select_single_area":null,"split_area_by_probe":true,"n_jobs":null,"use_process_pool":true,"max_workers":null,"units_group_by":["{\\"Column\\":\\"session_id\\"}","{\\"Column\\":\\"structure\\"}","{\\"Column\\":\\"electrode_group_name\\"}"],"units_query":"{\\"BinaryExpr\\

In [10]:
import json
pl.Expr.deserialize(io.StringI(json.loads(params.model_dump_json())['units_query'])

/tmp/ipykernel_162/3797388423.py:2: DeprecationWarning: `Expr.from_json` is deprecated. It has been renamed to `Expr.deserialize`. Note that the new method operates on file-like inputs rather than strings. Enclose your input in `io.StringIO` to keep the same behavior.
  pl.Expr.from_json(json.loads(params.model_dump_json())['units_query'])


<Expr ['[([([(col("isi_violations_rati…'] at 0x7F8B47CCEF20>

In [30]:
current = Params.model_validate_json(
    upath.UPath('s3://aind-scratch-data/dynamic-routing/ben/decoding/debug_20250327_165658/params.json').read_text()
)
current

Params(result_prefix='debug', session_id='703333_2024-04-10', run_id='20250327_165658', skip_existing=False, test=False, logging_level='INFO', update_packages_from_source=False, override_params_json='{}', session_table_query="is_ephys & is_task & is_annotated & is_production & issues=='[]'", unit_criteria='medium', n_units=20, n_repeats=25, input_data_type='spikes', vid_angle_facemotion='face', vid_angle_LP='behavior', central_section='4_blocks_plus', exclude_cue_trials=False, n_unit_threshold=5, keep_n_SVDs=500, LP_parts_to_keep=['ear_base_l', 'eye_bottom_l', 'jaw', 'nose_tip', 'whisker_pad_l_side'], spikes_binsize=0.2, spikes_time_before=0.2, spikes_time_after=0.01, use_structure_probe=True, crossval='5_fold', labels_as_index=True, decoder_type='LogisticRegression', only_use_all_units=False, predict='context', regularization=None, penalty=None, solver=None, select_single_area=None, split_area_by_probe=True, n_jobs=None, use_process_pool=True, max_workers=None)

In [38]:
new = Params(
    result_prefix='debug', session_id='703333_2024-04-10', run_id='20250327_165658', override_params_json='{}', skip_existing=True,
)
new.model_dump() == current.model_dump()

False

In [4]:
delta = pl.scan_delta(
    's3://aind-scratch-data/dynamic-routing/ben/decoding/debug_20250327_165658'
)

In [19]:
(
    delta
    .group_by('session_id')
    .agg(
        pl.col('structure').n_unique().alias('n_structures'),
    )
    .collect()
    .sort('n_structures')
)

session_id,n_structures
str,u32
"""662892_2023-08-21""",1
"""702136_2024-03-04""",1
"""668755_2023-08-31""",1
"""702136_2024-03-06""",1
"""712815_2024-05-22""",1
…,…
"""703880_2024-04-18""",7
"""733891_2024-09-18""",7
"""712815_2024-05-21""",7


In [12]:
(
    delta
    .sort('repeat_idx')
    # for each repeat, get mean accuracy across all shifts
    .group_by('session_id', 'structure', 'electrode_group_name', 'n_units', 'repeat_idx', maintain_order=True)
    .agg(
        pl.mean('balanced_accuracy_test').alias('mean_balanced_accuracy_test'),
        pl.col('balanced_accuracy_test').filter(pl.col('shift_idx').eq(0)).first().alias('balanced_accuracy_test_shift_0'),
    )
    # get mean of 0-shifts across repeats
    .group_by('session_id', 'structure', 'electrode_group_name', 'n_units')
    .agg(
        pl.all().exclude('balanced_accuracy_test_shift_0'),
        pl.mean('balanced_accuracy_test_shift_0').alias('mean_balanced_accuracy_test_shift_0'),
    )
    .collect()
)

session_id,structure,electrode_group_name,n_units,repeat_idx,mean_balanced_accuracy_test,mean_balanced_accuracy_test_shift_0
str,str,str,i32,list[i64],list[f64],f64
"""649943_2023-02-16""","""MGv""","""probeD""",20,"[0, 1, … 24]","[0.547665, 0.539835, … 0.540562]",0.585301
"""741137_2024-10-11""","""SCm""","""probeC""",20,"[0, 1, … 24]","[0.684017, 0.685999, … 0.659854]",0.765812
"""636766_2023-01-26""","""ILA""","""probeE""",20,"[0, 1, … 24]","[0.541578, 0.557304, … 0.56375]",0.591123
"""726088_2024-06-20""","""DP""","""probeF""",20,"[0, 1, … 24]","[0.620756, 0.621033, … 0.614592]",0.665033
"""702136_2024-03-05""","""LD""","""probeB""",20,"[0, 1, … 24]","[0.513912, 0.540432, … 0.53868]",0.56173
…,…,…,…,…,…,…
"""712815_2024-05-21""","""SMT""","""probeA""",20,"[0, 1, … 24]","[0.567256, 0.576199, … 0.581139]",0.638409
"""741137_2024-10-11""","""POST""","""probeD""",20,[0],[0.555036],0.574621
"""702131_2024-02-26""","""SSp""","""probeA""",20,"[0, 1, … 24]","[0.525138, 0.556091, … 0.537326]",0.583978


In [14]:
units = get_filtered_units(params).collect()

In [18]:
units = utils.get_df('units', lazy=False)

In [21]:
params.units_group_by

[<Expr ['col("session_id")'] at 0x7EFC096D8B80>,
 <Expr ['col("structure")'] at 0x7EFC096D9F90>,
 <Expr ['col("electrode_group_name")'] at 0x7EFC096D9720>]

In [31]:
params.units_query

<Expr ['[([([(col("isi_violations_rati…'] at 0x7EFC096DA080>

In [33]:
(
    units
    .pipe(group_structures, keep_originals=True)
    .filter(
        params.units_query,
        pl.col('isi_violations_ratio').le(0.5),
        pl.col('presence_ratio').ge(0.9),
        pl.col('amplitude_cutoff').le
    )
    .filter(
        pl.col('unit_id').n_unique().ge(params.n_units).over(params.units_group_by),
    )
    .group_by(*params.units_group_by)
    .agg(
        pl.col('unit_id').n_unique().alias('n_units'),
    )
    .sort('n_units')
)


session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""703333_2024-04-10""","""out of brain""","""probeB""",1
"""644864_2023-01-31""","""out of brain""","""probeA""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636397_2022-09-27""","""out of brain""","""probeF""",1
"""626791_2022-08-16""","""out of brain""","""probeC""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584


In [17]:
units.group_by(*params.units_group_by).agg(pl.col('unit_id').n_unique().alias('n_units')).sort('n_units')

session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""626791_2022-08-16""","""out of brain""","""probeC""",1
"""742903_2024-10-23""","""DG""","""probeE""",1
"""733780_2024-08-26""","""TTd""","""probeF""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636766_2023-01-25""","""out of brain""","""probeE""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584
